In [4]:
import warnings
warnings.filterwarnings('ignore')

In [128]:
import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, report, training, utils, Variable, computational_graph
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions

import tensorflow as tf

from edward.models import Normal as TNormal

# scipy

In [109]:
from scipy.stats import uniform, norm, poisson

In [120]:
poisson.logpmf(k=2, mu=1)

-1.6931471805599454

In [121]:
mu = Variable(np.array([1]).astype(np.float32), name="mu")
sigma = Variable(np.array([3]).astype(np.float32), name="sigma")

In [122]:
mu, sigma

(variable mu([1.]), variable sigma([3.]))

In [123]:
norm.logpdf(1.95, loc=mu.data, scale=sigma.data)

array([-2.06768974])

In [124]:
norm.rvs(loc=[0, 1], scale=[1, 2], size=(2, 2), random_state=None)

array([[-0.146666  ,  0.9098745 ],
       [ 0.03988563,  0.97242832]])

In [125]:
uniform.logpdf(1.95, loc=1, scale=2)

-0.6931471805599453

# PyTorch

In [126]:
import torch
from torch.autograd import Variable

import pyro
import pyro.distributions as dist

In [78]:
mu = Variable(torch.Tensor([1]))   # mean zero
sigma = Variable(torch.Tensor([3]))

x = Variable(torch.Tensor([1.95]))

In [80]:
dist.normal.log_pdf(x, mu, sigma)

Variable containing:
-2.0677
[torch.FloatTensor of size 1]

In [81]:
dist.uniform.log_pdf(x, mu, sigma)

Variable containing:
-0.6931
[torch.FloatTensor of size 1]

In [49]:
PNormal

torch.distributions.Normal

In [25]:
a = torch.autograd.Variable(torch.Tensor(0))
b = torch.autograd.Variable(torch.Tensor(2))

# Edward

In [161]:
a = TNormal(0.0, 1.0)
b = TNormal(a, 1.0)
c = TNormal(0.0, 1.0)
d = TNormal(b * c, 1.0)

In [169]:
b.get_parents()[0] == a

True

In [165]:
d.get_parents()

[<ed.RandomVariable 'Normal_7/' shape=() dtype=float32>,
 <ed.RandomVariable 'Normal_8/' shape=() dtype=float32>]

In [164]:
set(d.get_parents()) == set([b, c])

True

In [159]:
graph = tf.get_default_graph()

In [160]:
graph.get_all_collection_keys()

[]

In [156]:
help(tf.Graph)

Help on class Graph in module tensorflow.python.framework.ops:

class Graph(builtins.object)
 |  A TensorFlow computation, represented as a dataflow graph.
 |  
 |  A `Graph` contains a set of
 |  @{tf.Operation} objects,
 |  which represent units of computation; and
 |  @{tf.Tensor} objects, which represent
 |  the units of data that flow between operations.
 |  
 |  A default `Graph` is always registered, and accessible by calling
 |  @{tf.get_default_graph}.
 |  To add an operation to the default graph, simply call one of the functions
 |  that defines a new `Operation`:
 |  
 |  ```python
 |  c = tf.constant(4.0)
 |  assert c.graph is tf.get_default_graph()
 |  ```
 |  
 |  Another typical usage involves the
 |  @{tf.Graph.as_default}
 |  context manager, which overrides the current default graph for the
 |  lifetime of the context:
 |  
 |  ```python
 |  g = tf.Graph()
 |  with g.as_default():
 |    # Define operations and tensors in `g`.
 |    c = tf.constant(30.0)
 |    assert c

# Chainer

In [214]:
x = Variable(np.array(1), name="x")
y = x + 1
z = y + x + 1

In [244]:
x.__add__

<bound method add of variable x(1)>

In [215]:
type(y.creator.inputs[0].get_variable())

chainer.variable.Variable

In [216]:
[i.get_variable() for i in y.creator.inputs]

[variable x(1)]

In [222]:
z.node

In [221]:
z.node.creator.outputs

(<weakref at 0x10411c6d8; to 'VariableNode' at 0x122b26668>,)

In [225]:
import weakref

In [227]:
help(weakref)

Help on module weakref:

NAME
    weakref - Weak reference support for Python.

MODULE REFERENCE
    https://docs.python.org/3.6/library/weakref
    
    The following documentation is automatically generated from the Python
    source files.  It may be incomplete, incorrect or include features that
    are considered implementation detail and may vary between Python
    implementations.  When in doubt, consult the module reference at the
    location listed above.

DESCRIPTION
    This module is an implementation of PEP 205:
    
    http://www.python.org/dev/peps/pep-0205/

CLASSES
    builtins.object
        _weakrefset.WeakSet
        builtins.weakcallableproxy
        builtins.weakproxy
        builtins.weakref
            WeakMethod
        finalize
    collections.abc.MutableMapping(collections.abc.Mapping)
        WeakKeyDictionary
        WeakValueDictionary
    
    CallableProxyType = class weakcallableproxy(object)
     |  Methods defined here:
     |  
     |  __abs__(self

In [236]:
z.creator.outputs[0]()

In [235]:
help(z.creator.outputs[0])

Help on weakref object:

class weakref(object)
 |  Methods defined here:
 |  
 |  __call__(self, /, *args, **kwargs)
 |      Call self as a function.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __hash__(self, /)
 |      Return hash(self).
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __le__(self, value, /)
 |      Return self<=value.
 |  
 |  __lt__(self, value, /)
 |      Return self<value.
 |  
 |  __ne__(self, value, /)
 |      Return self!=value.
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.
 |  
 |  __repr__(self, /)
 |      Return repr(self).
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __callback__

In [201]:
help(chainer.variable.VariableNode)

Help on class VariableNode in module chainer.variable:

class VariableNode(builtins.object)
 |  Node in the backward computational graph representing a variable.
 |  
 |  This object represents a variable node in a computational graph. The node
 |  is used in error backpropagation (a.k.a. backprop) to determine which
 |  gradient to be passed to each function.
 |  
 |  A variable node is held by the corresponding :class:`~chainer.Variable`
 |  object, which is managed by users. :class:`~chainer.FunctionNode` objects
 |  that take the variable as an input also hold references to the variable
 |  node.
 |  
 |  Note that the node does not hold a reference to the corresponding data
 |  array in general. The data array is actually accessible by the node in the
 |  following cases.
 |  
 |  1. If there exists a :class:`~chainer.Variable` object that holds a
 |     reference to the variable node, the variable node holds a weak reference
 |     to the variable object, and thus the data array 

In [182]:
y.creator.outputs

(<weakref at 0x122b4a098; to 'VariableNode' at 0x122b3dc88>,)